<a href="https://colab.research.google.com/github/ii200400/Tensorflow_Tutorial/blob/master/10%20-%20RNN/02_Autocomplete(2_1ver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

자연어 처리나 음성 처리 분야에 많이 사용되는 RNN 의 기본적인 사용법을 익히자!

4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성하는 프로그램을 만들었다.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

## 데이터 정의

평소처럼 이미 있는 데이터 셋을 가져와서 하는 것이 아니라 임의로 데이터를 만들어서 훈련 및 테스트를 한다.

때문에 아주 제한적인 상황에서만 쓰일 수 있는 모델이 만들어지며 사용하는 데이터는 아래와 같다.

In [ ]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# one-hot 인코딩 및 디코딩을 사용 하기 위해 딕셔너리를 만든다.
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 다음 배열은 입력값과 출력값으로 아래와 같은 형식을 가진다.
# wor -> X, d -> Y
# woo -> X, d -> Y
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

### 데이터 인코딩

In [ ]:
def make_batch(seq_data):
  input_batch = []
  target_batch = []

  for seq in seq_data:
    # 여기서 생성하는 input_batch 와 target_batch 는
    # 알파벳 배열의 인덱스 번호이다.
    # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
    input = [num_dic[n] for n in seq[:-1]]

    # 3, 3, 15, 4, 3 ...
    target = num_dic[seq[-1]]

    # one-hot 인코딩을 한다.
    # input 이 [0, 1, 2] 라면
    # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
    #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
    #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
    # numpy-array[리스트] 와 같은 형식은 아래의 셀에 예시를 만들어 놓았다.
    input_batch.append(np.eye(dic_len)[input])

    # 지금까지 손실함수로 사용하던 softmax_cross_entropy_with_logits 함수는
    # label 값을 one-hot 인코딩으로 넘겨줘야 하지만,
    # 이 예제에서 사용할 손실 함수인 sparse_softmax_cross_entropy_with_logits 는
    # one-hot 인코딩을 사용하지 않으므로 index 로 바로 넘긴다.
    target_batch.append(target)

  return np.array(input_batch), np.array(target_batch)

## 모델 구성

### 옵션 설정

In [ ]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

# RNN 을 구성하는 시퀀스의 갯수 (문자 내 알파벳 갯수)
n_step = 3

# 입력값과 출력값의 크기는
# 알파벳에 대한 one-hot 인코딩이므로 26개가 된다.
# 예) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
n_input = n_class = dic_len

### 신경망 구성

In [ ]:
def make_model():
  # 과적합 방지를 위해 Dropout 기법을 사용한 RNN 셀을 생성한다.
  cell1 = tf.keras.layers.LSTMCell(n_hidden, dropout=0.5)
  # 여러개의 셀을 조합해서 사용하기 위해 셀을 추가로 생성
  cell2 = tf.keras.layers.LSTMCell(n_hidden)
  # 위에서 만든 셀들을 조합한 RNN 셀을 생성한다.
  multi_cell = tf.keras.layers.StackedRNNCells([cell1, cell2])

  model = tf.keras.models.Sequential([
    # 함수를 이용해 순환 신경망을 만든다.
    tf.keras.layers.RNN(multi_cell, dtype=tf.float32, input_shape=(n_step, n_input)),
    tf.keras.layers.Dense(n_class, activation='softmax')
  ])

  return model

model = make_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, 128)               210944    
_________________________________________________________________
dense (Dense)                (None, 26)                3354      
Total params: 214,298
Trainable params: 214,298
Non-trainable params: 0
_________________________________________________________________


### 비용 및 최적화

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])

### 모델 학습

In [ ]:
input_batch, target_batch = make_batch(seq_data)

model.fit(x=input_batch,
          y=target_batch,
          epochs=total_epoch)

Epoch 1/30
1/1 [==============================] - 0s 2ms/step - loss: 3.2626 - accuracy: 0.1000
Epoch 2/30
1/1 [==============================] - 0s 1ms/step - loss: 3.1631 - accuracy: 0.6000
Epoch 3/30
1/1 [==============================] - 0s 1ms/step - loss: 2.9328 - accuracy: 0.5000
Epoch 4/30
1/1 [==============================] - 0s 1ms/step - loss: 2.3951 - accuracy: 0.6000
Epoch 5/30
1/1 [==============================] - 0s 1ms/step - loss: 1.9491 - accuracy: 0.5000
Epoch 6/30
1/1 [==============================] - 0s 1ms/step - loss: 1.7874 - accuracy: 0.5000
Epoch 7/30
1/1 [==============================] - 0s 1ms/step - loss: 1.7364 - accuracy: 0.5000
Epoch 8/30
1/1 [==============================] - 0s 1ms/step - loss: 1.1798 - accuracy: 0.6000
Epoch 9/30
1/1 [==============================] - 0s 1ms/step - loss: 1.3351 - accuracy: 0.3000
Epoch 10/30
1/1 [==============================] - 0s 1ms/step - loss: 1.2715 - accuracy: 0.4000
Epoch 11/30
1/1 [======================

### 결과 확인

In [ ]:
# 레이블값이 정수이므로 예측값도 정수로 출력하도록 한다.
predict = tf.argmax(model(input_batch), 1)
# 정수인 입력값을 비교한다.
prediction_check = tf.equal(prediction, target_batch)
accuracy_val = tf.reduce_mean(tf.cast(prediction_check, tf.float32)).numpy()

predict_words = []
for idx, val in enumerate(seq_data):
  last_char = char_arr[predict[idx]]
  predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cood', 'load', 'love', 'kiss', 'kind']
정확도: 0.9


## 전체 코드

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

#########
# 데이터 정의
######

char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

def make_batch(seq_data):
  input_batch = []
  target_batch = []

  for seq in seq_data:
    input = [num_dic[n] for n in seq[:-1]]
    target = num_dic[seq[-1]]

    input_batch.append(np.eye(dic_len)[input])
    target_batch.append(target)

  return np.array(input_batch), np.array(target_batch)

#########
# 옵션 설정
######

learning_rate = 0.01
n_hidden = 128
total_epoch = 30

n_step = 3

n_input = n_class = dic_len

#########
# 신경망 모델 구성
######

def make_model():
  cell1 = tf.keras.layers.LSTMCell(n_hidden, dropout=0.5)
  cell2 = tf.keras.layers.LSTMCell(n_hidden)
  multi_cell = tf.keras.layers.StackedRNNCells([cell1, cell2])

  model = tf.keras.models.Sequential([
    tf.keras.layers.RNN(multi_cell, dtype=tf.float32, input_shape=(n_step, n_input)),
    tf.keras.layers.Dense(n_class, activation='softmax')
  ])

  return model

model = make_model()
# model.summary()

model.compile(optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])

#########
# 신경망 모델 학습
######

input_batch, target_batch = make_batch(seq_data)

model.fit(x=input_batch,
          y=target_batch,
          epochs=total_epoch)

#########
# 결과 확인
######

predict = tf.argmax(model(input_batch), 1)
prediction_check = tf.equal(prediction, target_batch)
accuracy_val = tf.reduce_mean(tf.cast(prediction_check, tf.float32)).numpy()

predict_words = []
for idx, val in enumerate(seq_data):
  last_char = char_arr[predict[idx]]
  predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)